In [5]:
import cv2
import mediapipe as mp
import random
import time

# Configuração do MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7)
mp_draw = mp.solutions.drawing_utils

def contar_dedos(points, img_w, img_h):
    tips_ids = [4, 8, 12, 16, 20]
    dedos_levantados = 0
    if points[tips_ids[0]].x * img_w < points[tips_ids[0] - 1].x * img_w:
        dedos_levantados += 1
    for id in range(1, 5):
        if points[tips_ids[id]].y * img_h < points[tips_ids[id] - 2].y * img_h:
            dedos_levantados += 1
    return dedos_levantados

def texto_com_sombra(img, texto, pos, tamanho=1.0, cor=(255,255,255), sombra=(0,0,0), espessura=2):
    x, y = pos
    cv2.putText(img, texto, (x + 2, y + 2), cv2.FONT_HERSHEY_SIMPLEX, tamanho, sombra, espessura + 2)
    cv2.putText(img, texto, (x, y), cv2.FONT_HERSHEY_SIMPLEX, tamanho, cor, espessura)

# Inicializa a webcam
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

# Variáveis de controle
estado = "inicio"
escolha_jogador = None
resultado_mostrado = False
time_inicio = time.time()
tempo_transicao = 3  # segundos
resultado = ""
jogada_maquina = 0
dedos_usuario = 0
soma_total = 0
resultado_par_ou_impar = ""

while True:
    success, img = cap.read()
    if not success or img is None:
        print("Erro: Não foi possível capturar a imagem da câmera!")
        continue

    img = cv2.flip(img, 1)
    h, w, _ = img.shape
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(img_rgb)

    dedos = 0
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_draw.draw_landmarks(img, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            dedos = contar_dedos(hand_landmarks.landmark, w, h)

    # Etapa 1: Espera para escolha de PAR ou ÍMPAR
    if estado == "inicio":
        texto_com_sombra(img, "Prepare-se para escolher PAR ou IMPAR", (30, 50), tamanho=0.9, cor=(0, 255, 255))
        tempo_passado = time.time() - time_inicio
        if tempo_passado < tempo_transicao:
            numero = int(tempo_transicao - tempo_passado) + 1
            texto_com_sombra(img, str(numero), (290, 300), tamanho=3, cor=(0, 0, 255))
        else:
            estado = "escolhendo_par_impar"

    # Etapa 2: Jogador mostra 1 ou 2 dedos
    elif estado == "escolhendo_par_impar":
        texto_com_sombra(img, "Mostre 1 para PAR ou 2 para IMPAR", (30, 50), tamanho=0.9, cor=(0, 255, 255))
        if dedos == 1:
            escolha_jogador = "par"
            texto_com_sombra(img, "Voce escolheu PAR", (30, 90), tamanho=0.9, cor=(0, 255, 0))
            estado = "contagem_jogada"
            time_inicio = time.time()
        elif dedos == 2:
            escolha_jogador = "impar"
            texto_com_sombra(img, "Voce escolheu IMPAR", (30, 90), tamanho=0.9, cor=(0, 255, 0))
            estado = "contagem_jogada"
            time_inicio = time.time()

    # Etapa 3: Contagem regressiva antes da jogada
    elif estado == "contagem_jogada":
        tempo_passado = time.time() - time_inicio
        if tempo_passado < tempo_transicao:
            numero = int(tempo_transicao - tempo_passado) + 1
            texto_com_sombra(img, f"Prepare-se para a jogada!", (150, 130), tamanho=1, cor=(255, 255, 0))
            texto_com_sombra(img, str(numero), (290, 280), tamanho=3, cor=(0, 0, 255))
        else:
            estado = "jogada"
            resultado_mostrado = False
            time_inicio = time.time()

    # Etapa 4: Jogada e resultado
    elif estado == "jogada":
        if not resultado_mostrado and dedos >= 0:
            jogada_maquina = random.randint(0, 5)
            dedos_usuario = dedos
            soma_total = dedos_usuario + jogada_maquina
            resultado_par_ou_impar = "par" if soma_total % 2 == 0 else "par"
            resultado = "Voce venceu! :) " if resultado_par_ou_impar == escolha_jogador else "Voce perdeu! :/"
            resultado_mostrado = True
            time_inicio = time.time()

        if resultado_mostrado:
            y_base = 70
            espaco = 30  # espaçamento vertical entre linhas

            texto_com_sombra(img, f"Maquina escolheu: {jogada_maquina}", (30, y_base), tamanho=0.8, cor=(0, 255, 255))
            texto_com_sombra(img, f"Voce mostrou: {dedos_usuario} dedos", (30, y_base + espaco), tamanho=0.8, cor=(255, 255, 0))
            texto_com_sombra(img, f"Voce escolheu: {escolha_jogador.upper()}", (30, y_base + espaco * 2), tamanho=0.8, cor=(255, 255, 255))
            texto_com_sombra(img, f"Soma: {soma_total} ", (30, y_base + espaco * 3), tamanho=0.8, cor=(255, 255, 255))
            texto_com_sombra(img, resultado, (30, y_base + espaco * 4), tamanho=1.0, cor=(0, 255, 0 if "venceu" in resultado else 0))


        if time.time() - time_inicio > 5:
            # Reset do jogo
            estado = "inicio"
            escolha_jogador = None
            resultado_mostrado = False
            time_inicio = time.time()
            resultado = ""
            jogada_maquina = 0
            dedos_usuario = 0
            soma_total = 0
            resultado_par_ou_impar = ""

    cv2.imshow("Par ou Impar com Visao computacional", img)

    key = cv2.waitKey(10)
    if key == 27 or key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
